In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

### **Data Reading**

In [0]:
df = spark.read.parquet("abfss://bronze@dlsdatabrickseteteamea.dfs.core.windows.net/products")

In [0]:
df.display()

In [0]:
df = df.drop("_rescued_data")

### **Create Temp View**

In [0]:
df.createOrReplaceTempView("products")

### **Functions**

In [0]:
%sql
/* การเขียน Function ใน SQL หลักๆแล้วมี 3 แบบ คือ 
1. Scalar Function = รับ parameter มาค่าเดียว และ Return เพียงค่าเดียว (ชนิดข้อมูลแทบทุกชนิด ที่ไม่ใช่ชนิด Table)
2. Aggregate Function = รับ parameter แบบหลายค่า แต่ Return เพียงค่าเดียว (ชนิดข้อมูลแทบทุกชนิด ที่ไม่ใช่ชนิด Table)
3. Table Value Function = Function ที่ Return เป็น Table
*/

-- สร้าง Scalar Function ด้วย SQL
CREATE OR REPLACE FUNCTION databricks_cata.bronze.discount_func(p_price DOUBLE)
RETURNS DOUBLE
LANGUAGE SQL
RETURN p_price * 0.90


**Call the function with SQL**

In [0]:
%sql
SELECT 
  product_id, 
  price, 
  databricks_cata.bronze.discount_func(price) as discounted_price 
FROM products


**Call the function with PySpark**

In [0]:
# ถ้าจะ Call Function ใน SQL ให้เรียกใช้ฟังก์ชั่น expr(<catalog_name>.<schema_name>.<function_name>(<column_name>))
df = df.withColumn("discounted_price", expr("databricks_cata.bronze.discount_func(price)"))
df.display()

In [0]:
%sql
CREATE OR REPLACE FUNCTION databricks_cata.bronze.upper_func(p_string STRING)
RETURNS STRING
LANGUAGE PYTHON
AS 
$$
  return p_string.upper()
$$

-- ถ้าเราจะเขียนโค้ดด้วย Python ที่จะถูกใช้ในฟังก์ชัน SQL เราจะเป็นต้องระบุ $$ ไว้ส่วนหัวและท้ายเพือระบุขอบเขตของภาษา python -- การใช้ $$ เป็นการบอกให้ SQL interpreter รู้ว่าข้อความระหว่าง $$ นั้นเป็นโค้ดของภาษาอื่น (ในที่นี้คือ Python) และไม่ใช่โค้ด SQL โดยตรง

In [0]:
%sql
-- df.withColumn("upper_brand", expr("databricks_cata.bronze.upper_func(brand)")).display()
SELECT product_id, brand, databricks_cata.bronze.upper_func(brand) as brand_upper FROM products


In [0]:
df.write.format("delta") \
    .mode("append") \
    .option("path", "abfss://silver@dlsdatabrickseteteamea.dfs.core.windows.net/products") \
    .save()